# Sqlite3 vs Shelve vs Numpy

This note is a toy model to measure the performance cost of Sqlite3 and Shelve databases.

For each sqlite3 or shelve database, it will only contain one member (a list of numpy arrays). 

Specifially, this note will measure the cost to call the database's member multiple times, i.e. 1000x.

To make it more interesting, I will also add a benchmark for `np.save`.

In [1]:
import torch
import numpy as np
import io
import time
import shelve
import sqlite3

In [2]:
s = 219
i, d = 64, 30
x = np.random.random_sample([i, d])
dx = np.random.random_sample([i, i, d, 3])
rdx = np.random.random_sample([i, d, 6])

print(x[30, :10])
print(dx[8, 3, :10, 2])
print(rdx[8, :10, 4])

[0.18392743 0.40013709 0.87024558 0.18679166 0.86769028 0.90518177
 0.80863903 0.27152487 0.40435877 0.77129857]
[0.39939181 0.72122024 0.98375967 0.32827872 0.30841818 0.36752987
 0.99104522 0.72557409 0.44660145 0.10458994]
[0.53994022 0.40873679 0.68797249 0.26424826 0.23271192 0.4937447
 0.42121443 0.31617132 0.93257187 0.2151361 ]


In [3]:
### Shelve ###

# Insert
db = shelve.open("database")
t0 = time.time()
for i in range(s):
    db[str(i)] = [x, dx, rdx]
t1 = time.time()
db.close()

# Reading
db = shelve.open("database")
_t0 = time.time()
for i in range(s):
    _d = db[str(i)]
    _x = _d[0]
    _dx = _d[1]
    _rdx = _d[2]
_t1 = time.time()
db.close()

print(_x[30, :10])
print(_dx[8, 3, :10, 2])
print(_rdx[8, :10, 4])
print("The time it takes to save shelve database:", t1-t0, "s")
print("The time it takes to load shelve database:", _t1-_t0, "s")

[0.18392743 0.40013709 0.87024558 0.18679166 0.86769028 0.90518177
 0.80863903 0.27152487 0.40435877 0.77129857]
[0.39939181 0.72122024 0.98375967 0.32827872 0.30841818 0.36752987
 0.99104522 0.72557409 0.44660145 0.10458994]
[0.53994022 0.40873679 0.68797249 0.26424826 0.23271192 0.4937447
 0.42121443 0.31617132 0.93257187 0.2151361 ]
The time it takes to save shelve database: 0.5499148368835449 s
The time it takes to load shelve database: 0.1622021198272705 s


In [4]:
### Sqlite3 ###

def adapt_array(arr):
    return arr.tobytes()

def convert_array(text):
    return np.frombuffer(text)

# Converts np.array to TEXT when inserting
sqlite3.register_adapter(np.ndarray, adapt_array)

# Converts TEXT to np.array when selecting
sqlite3.register_converter("array", convert_array)

conn = sqlite3.connect("sqldatabase.db", detect_types=sqlite3.PARSE_DECLTYPES)
c = conn.cursor()
c.execute("create table test (num integer, x array, dx array, rdx array)")

# Insert
t0 = time.time()
for i in range(s):
    c.execute("insert into test (num, x, dx, rdx) values (?, ?, ?, ?)", (i, x, dx, rdx))
t1 = time.time()

# Reading
_t0 = time.time()
for i in range(s):
    c.execute("select * from test where num=?", (i,))
    data = c.fetchone()
    _x = np.reshape(data[1], x.shape)
    _dx = np.reshape(data[2], dx.shape)
    _rdx = np.reshape(data[3], rdx.shape)
_t1 = time.time()
print(_x[30, :10])
print(_dx[8, 3, :10, 2])
print(_rdx[8, :10, 4])
print("The time it takes to save sqlite3 database:", t1-t0, "s")
print("The time it takes to load sqlite3 database:", _t1-_t0, "s")

[0.18392743 0.40013709 0.87024558 0.18679166 0.86769028 0.90518177
 0.80863903 0.27152487 0.40435877 0.77129857]
[0.39939181 0.72122024 0.98375967 0.32827872 0.30841818 0.36752987
 0.99104522 0.72557409 0.44660145 0.10458994]
[0.53994022 0.40873679 0.68797249 0.26424826 0.23271192 0.4937447
 0.42121443 0.31617132 0.93257187 0.2151361 ]
The time it takes to save sqlite3 database: 0.5620255470275879 s
The time it takes to load sqlite3 database: 0.24424338340759277 s


In [5]:
### Numpy ###

data = {'x': x, 'dx': dx, 'rdx': rdx}
_data = []
for i in range(s):
    _data.append(data)

np.save("database.npy", _data)

t0 = time.time()
for i in range(s):
    data = np.load("database.npy", allow_pickle=True)
    _x = data[i]['x']
    _dx = data[i]['dx']
    _rdx = data[i]['rdx']
t1 = time.time()

print(_x[30, :10])
print(_dx[8, 3, :10, 2])
print(_rdx[8, :10, 4])
print("The time it takes to load numpy database:", t1-t0, "s")

[0.18392743 0.40013709 0.87024558 0.18679166 0.86769028 0.90518177
 0.80863903 0.27152487 0.40435877 0.77129857]
[0.39939181 0.72122024 0.98375967 0.32827872 0.30841818 0.36752987
 0.99104522 0.72557409 0.44660145 0.10458994]
[0.53994022 0.40873679 0.68797249 0.26424826 0.23271192 0.4937447
 0.42121443 0.31617132 0.93257187 0.2151361 ]
The time it takes to load numpy database: 0.13050246238708496 s


In [6]:
### Numpy Alt ###

_x, _dx, _rdx = [], [], []
for i in range(s):
    _x.append(x)
    _dx.append(dx)
    _rdx.append(rdx)

#np.save("x.npy", _x)
np.save("dx.npy", _dx)
#np.save("rdx.npy", _rdx)

t0 = time.time()
for i in range(s):    
    #_x = np.load("x.npy", allow_pickle=True, mmap_mode='r')
    _dx = np.load("dx.npy", allow_pickle=True, mmap_mode='r')
    #_rdx = np.load("rdx.npy", allow_pickle=True, mmap_mode='r')
t1 = time.time()

print(_dx[0][8, 3, :10, 2])
print("The time it takes to load numpy database:", t1-t0, "s")

[0.39939181 0.72122024 0.98375967 0.32827872 0.30841818 0.36752987
 0.99104522 0.72557409 0.44660145 0.10458994]
The time it takes to load numpy database: 0.04536080360412598 s


### Conclusion

Sqlite3 performs the worst because it doesn't store numpy array. Therefore, sqlite3 needs to bypass it by using external function to convert numpy array to byte and vice versa everytime the database is called.

It seems like numpy database provides the best performance for storing numpy array. Numpy storing can be further improved by `mmap_mode`.